In [ ]:
import networkx as nx
import random
import numpy as np

# Load the Email-Eu-Core dataset
G = nx.read_edgelist("C:\MTech\Second Semester\Web and Social Computing(IT752)\Project\email-Eu-core.txt")
G1 = nx.read_edgelist("C:\MTech\Second Semester\Web and Social Computing(IT752)\Project\email-Eu-core.txt")
# Choose some seed nodes from the dataset as input
#seeds = ['500', '2500', '9000', '25000', '50000', '80000', '125000', '160000', '200000', '245000']


# candidates is a list of data points
# num_seeds is the desired number of seed points
# set the random seed to ensure reproducibility
#random.seed(42)
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print(num_nodes)
print(num_edges)
seeds = random.sample(list(G.nodes), 15)
print(seeds)

# Define the number of hash functions
num_hashes = 400

# Define the number of bands
num_bands = 100

# Define the number of rows per band
rows_per_band = num_hashes // num_bands

# Define the prime number for hashing
prime = 4294967311

# Define the maximum hash value
max_hash = 2**32 - 1

# Define the minhash signature function
def minhash_signature(node, num_hashes):
    # Generate a random permutation of the node neighbors
    neighbors = list(G.neighbors(node))
    random.shuffle(neighbors)

    # Define the hash functions
    def hash_function(a, b, x):
        return ((a * x + b) % prime) % max_hash

    # Generate the minhash signature
    signature = []
    for i in range(num_hashes):
        min_hash = float('inf')
        for neighbor in neighbors:
            hash_value = hash_function(i+1, i+1, int(neighbor))
            if hash_value < min_hash:
                min_hash = hash_value
        signature.append(min_hash)
    return signature

# Define the LSH function
def LSH(seed, candidates, num_hashes, num_bands, rows_per_band):
    # Generate the minhash signatures for the seed and candidate nodes
    seed_signature = minhash_signature(seed, num_hashes)
    candidate_signatures = [minhash_signature(candidate, num_hashes) for candidate in candidates]

    # Define the bands
    bands = []
    for i in range(num_bands):
        band = []
        for j in range(rows_per_band):
            band.append(i * rows_per_band + j)
        bands.append(band)

    # Define the hash table
    hash_table = {}
    for i, candidate_signature in enumerate(candidate_signatures):
        for band in bands:
            key = tuple(candidate_signature[j] for j in band)
            if key not in hash_table:
                hash_table[key] = []
            hash_table[key].append(i)

    # Find the candidates that are near to the seed
    near_candidates = set()
    for band in bands:
        key = tuple(seed_signature[j] for j in band)
        if key in hash_table:
            near_candidates.update(hash_table[key])
    return near_candidates

#Following set is going to have all candidate values that are related to atleast one of the input seeds
absolute_candidate_set = set()
list_of_candidate_sets = []
# Apply the LSH function to each input seed node
for seed in seeds:
    candidates = list(G.nodes())
    candidates.remove(seed)
    near_candidates = LSH(seed, candidates, num_hashes, num_bands, rows_per_band)
    list_of_candidate_sets.append(near_candidates)
    absolute_candidate_set = absolute_candidate_set | near_candidates
    print(f"Seed node: {seed}")
    print(f"Near candidates: {near_candidates}")
    
print("\n")
print(absolute_candidate_set)
print("number of all total candidates = " + str(len(absolute_candidate_set)))

In [ ]:
from sklearn.cluster import AgglomerativeClustering

def select_related_candidates(seeds, candidates, k):
    """
    Select the k candidates that are most associated with the seed set
    
    Parameters:
    - seeds: set
        The set of seed accounts
    - candidates: list of sets
        The list of candidate accounts returned by LSH
    - k: int
        The number of candidates to select
        
    Returns:
    - selected_candidates: list of sets
        The k candidates that are most associated with the seed set
    """
    # Compute the Jaccard similarity score between each candidate and the seed set
    scores = []
    for candidate in candidates:
        score = len(candidate.intersection(seeds)) / len(candidate.union(seeds))
        scores.append(score)

    # Perform agglomerative clustering on the candidate set based on the similarity scores
    clustering = AgglomerativeClustering(n_clusters=k, affinity='precomputed', linkage='complete')
    similarity_matrix = np.array([[1 - score for score in scores]] * len(scores))
    clustering.fit(similarity_matrix)

    # Get the indices of the candidates in the largest cluster
    cluster_sizes = [list(clustering.labels_).count(i) for i in range(k)]
    largest_cluster_index = cluster_sizes.index(max(cluster_sizes))
    largest_cluster_indices = [i for i in range(len(clustering.labels_)) if clustering.labels_[i] == largest_cluster_index]

    # Select the candidates in the largest cluster
    selected_candidates = [candidates[i] for i in largest_cluster_indices]
  
    
    return selected_candidates


counter = 0
list_of_union = list()
list_after_Agglomerative_Clustering = select_related_candidates(seeds, list_of_candidate_sets, 1)
for candidate in list_after_Agglomerative_Clustering:
        print(candidate)
        list_of_union = list(set().union(list_of_union, candidate))
        counter = counter + 1
        if counter == 10:
            print()
            counter = 0
        
print("nodes after clustering = " + str(len(list_after_Agglomerative_Clustering[0])))

"""Working after agglomerative clustering"""
nodes_to_keep = list(list_after_Agglomerative_Clustering[0])
# nodes_to_keep = list_after_Agglomerative_Clustering[0]

# nodes_to_keep = set(nodes_to_keep)
print("list of union of all candidate sets----------")
print(list_of_union)

In [ ]:
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
import matplotlib.pyplot as plt
from networkx.algorithms.community import modularity

Gnew = nx.read_edgelist("C:\MTech\Second Semester\Web and Social Computing(IT752)\Project\email-Eu-core.txt")

Gnew = nx.convert_node_labels_to_integers(Gnew)
approx_neighbors = set()
approx_neighbors = list_of_union

Gnewsub = G.subgraph(list(approx_neighbors))

print(Gnewsub)

subgraph_to_consider = nx.DiGraph()

# Iterate over the edges in the original graph
for u, v in Gnew.edges():
    #print("out")
    #print("u = " + u + ", v = " + v)
    # If both nodes are in the set of nodes to include, add the edge to the subgraph
    if int(u) in approx_neighbors and int(v) in approx_neighbors:
        subgraph_to_consider.add_edge(u, v)
        #print("in")

print(subgraph_to_consider)
communities = list(greedy_modularity_communities(subgraph_to_consider))

# Assign a different color to each community
colors = ["r"]
color_map = {}
for i, comm in enumerate(communities):
    for node in comm:
        color_map[node] = colors[i % len(colors)]

# Draw the graph with each community shown in a different color
nx.draw_networkx(subgraph_to_consider, node_color=[color_map[node] for node in subgraph_to_consider.nodes()], node_size=20, with_labels=False)
plt.show()


In [ ]:
def jaccard_distance(G, node1, node2):
    succ1 = set(G.successors(node1))
    succ2 = set(G.successors(node2))
    
    intersection = len(succ1.intersection(succ2))
    union = len(succ1.union(succ2))
    
    if union == 0:
        return 0.0  # Avoid division by zero error
    
    jaccard_index = intersection / float(union)
    #jaccard_dist = 1 - jaccard_index
    
    return jaccard_index

nodes = subgraph_to_consider.nodes()

for i in nodes:
    for j in nodes:
        print("Jaccard similarity for",i, "and", j, "is :", jaccard_distance(subgraph_to_consider, i, j))


In [ ]:
import networkx as nx
import numpy as np
from scipy.spatial import distance
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt

# Calculate the Jaccard distance matrix
jac = np.zeros((subgraph_to_consider.number_of_nodes(), subgraph_to_consider.number_of_nodes()))
for i, u in enumerate(subgraph_to_consider.nodes()):
    for j, v in enumerate(subgraph_to_consider.nodes()):
        if i != j:
            successors_u = set(subgraph_to_consider.successors(u))
            successors_v = set(subgraph_to_consider.successors(v))
            if len(successors_u.union(successors_v)) > 0:
                jac[i, j] = len(successors_u.intersection(successors_v)) / len(successors_u.union(successors_v))

# Convert the matrix into a graph
jac_graph = nx.from_numpy_matrix(jac)

# convert the Jaccard similarity matrix into a distance matrix
distance_matrix = np.sort(distance.squareform(1 - jac, checks=False))

# generate the hierarchical clustering and dendrogram
linkage_matrix = hierarchy.linkage(distance_matrix, method='average')
plt.figure(figsize=(25,25))
dendrogram = hierarchy.dendrogram(linkage_matrix, labels=list(subgraph_to_consider.nodes()))
plt.show()

In [ ]:
from cdlib.algorithms import louvain
from cdlib import evaluation
g = subgraph_to_consider
UG = g.to_undirected()
communities = (louvain(UG))
mod = evaluation.conductance(UG,communities)
print(mod)

In [ ]:
S = set(subgraph_to_consider.nodes())
conductance = 1.0
for v in subgraph_to_consider.nodes():
    Sv = set(subgraph_to_consider.neighbors(v))
    if len(Sv) == 0:
        continue
    conductance_v = len(Sv - set([v])) / min(len(Sv), len(S - Sv))
    conductance = min(conductance, conductance_v)

print(f"The conductance ratio of the graph is {conductance}")


In [ ]:
# calculate cohesiveness
n = subgraph_to_consider.number_of_nodes()
m = subgraph_to_consider.number_of_edges()
k = (n*(n-1))/2  # maximum number of edges for a complete graph
cohesiveness = m / k

# calculate density
density = nx.density(subgraph_to_consider)

print("Cohesiveness:", cohesiveness)
print("Density:", density)

In [ ]:
# calculate the edge betweenness centrality
ebc = nx.edge_betweenness_centrality(subgraph_to_consider)

# sort the edges by their betweenness centrality
sorted_ebc = sorted(ebc.items(), key=lambda x: x[1], reverse=True)

# calculate the separability of the graph
separability = 0
for e, w in sorted_ebc:
    # remove the edge with highest betweenness centrality
    subgraph_to_consider.remove_edge(*e)
    # check if the graph is strongly connected
    if not nx.is_strongly_connected(subgraph_to_consider):
        separability = w
        break

# print the separability of the graph
print("Separability of the graph:", separability)